In [1]:
import pandas as pd
import datetime

In [2]:
legislative_period_19_begin = pd.Timestamp("2017-10-24", tz="UTC").replace(microsecond=0)
legislative_period_19_end = pd.Timestamp("2021-10-26", tz="UTC").replace(microsecond=0)

In [3]:
twitter_users = pd.read_csv('..\\data\\DE_politician_twitter_users.csv')
mps = pd.read_excel('..\\data\\MPs_WP19_2.xlsx',index_col=0)

In [4]:
#cast twitter users 'created_at' to date
twitter_users['created_at'] = pd.to_datetime(twitter_users['created_at'])
twitter_users = twitter_users[twitter_users['created_at'] < legislative_period_19_end]

In [5]:
twitter_users['start'] = [pd.NA]*len(twitter_users)
twitter_users['end'] = [pd.NA]*len(twitter_users)

In [6]:
twitter_users.head()

,author_id,username,name,description,followers_count,following_count,tweet_count,created_at,party,start,end
0,1419751204465356805,FBsirske,Frank Bsirske,Mitglied des 20. deutschen Bundestages; Gewerk...,2011,501,144,2021-07-26 20:08:02+00:00,Alliance 90/The Greens,<NA>,<NA>
1,1416388709264633858,HenningOtteMdB,Henning Otte,stellv. Vorsitzender des Verteidigungsausschus...,930,164,133,2021-07-17 13:26:19+00:00,CDU/CSU,<NA>,<NA>
2,1413536612068245507,TakisMehmetAli,Takis Mehmet Ali,"Deutsch-Grieche, Bundestagsabgeordneter (SPD) ...",1181,135,33,2021-07-09 16:33:24+00:00,SPD,<NA>,<NA>
3,1411676203224870912,goerke_c,Christian Görke,"Mitglied des Bundestages @linksfraktion, WK 64...",1624,986,408,2021-07-04 13:20:31+00:00,The Left Party,<NA>,<NA>
4,1410671503679574016,laura_kraft_,"Laura Kraft, MdB🌻",🏛 Bundestagsabgeordnete für Siegen-Wittgenstei...,823,236,120,2021-07-01 18:48:34+00:00,Alliance 90/The Greens,<NA>,<NA>


In [7]:
#match all the mps of the 19th legislative period with the twitter users

#get the index of the matching names
def get_index(x):
    return twitter_users[twitter_users['name'] == x].index.values.astype(int)
    
#match the names and set start/end date
for r in mps.index:
    idx = get_index(mps.loc[r,'VORNAME']+' '+mps.loc[r,'NACHNAME'])
    if len(idx) == 0:
        names = twitter_users[twitter_users["name"].str.contains(mps.loc[r,'VORNAME'])]
        idx_match = names[names["name"].str.contains(mps.loc[r,'NACHNAME'])].index
        if len(idx_match) == 0:
            print(mps.loc[r,'NAME'])
        else:
            twitter_users.loc[idx_match,'start'] = mps.loc[r, 'START']
            twitter_users.loc[idx_match,'end'] = mps.loc[r, 'END']


    else:
        twitter_users.loc[idx,'start'] = mps.loc[r, 'START']
        twitter_users.loc[idx,'end'] = mps.loc[r, 'END']

Peter Bleser
Dr. Eberhard Brecht
Hans-Joachim Fuchtel
Volker Kauder
Wolfgang Kubicki
Katharina Landgraf
Dr. Angela Merkel
Dr. Peter Ramsauer
Christian Schmidt
Ulla Schmidt
Dr. Hermann Otto Solms
Michael Stübgen
Doris Barnett
Dagmar Freitag
Manfred Grund
Dr. h. c. (Univ Kyiv) Hans Michelbach
Dr. Gerd Müller
Johannes Selle
Max Straubinger
Norbert Barthle
Dr. Ralf Brauksiepe
Marie-Luise Dött
Axel E. Fischer
Dr. Hans-Peter Friedrich
Martin Hohmann
Johannes Kahrs
Christine Lambrecht
Andrea Nahles
Kersten Steinke
Dietmar Nietan
René Röspel
Claudia Roth
Anita Schäfer
Ewald Schurer
Arnold Vaatz
Annette Widmann-Mauz
Klaus-Peter Willsch
Ingrid Arndt-Brauer
Dr. Carola Reimann
Sören Bartol
Veronika Bellmann
Alexander Dobrindt
Enak Ferlemann
Dr. Maria Flachsbarth
Otto Fricke
Eberhard Gienger
Michael Grosse-Brömer
Bettina Hagedorn
Anja Hajduk
Gabriele Hiller-Ohm
Patricia Lips
Caren Marks
Stephan Mayer
Marlene Mortler
Dr. Rolf Mützenich
Dr. Georg Nüßlein
Friedrich Ostendorff
Dr. Joachim Pfeiffer
Dr. 

In [8]:
twitter_users_19 = twitter_users[twitter_users['start'].isnull() == False]

In [9]:
twitter_users_19.to_csv('..\\data\\twitter_mps_wp19.csv', index=False)